In [124]:
from datetime import datetime, timedelta
import yfinance as yf   #https://pypi.org/project/yfinance/


In [125]:
# turn off debug to get data
debug = False

In [126]:
%run merge-data.ipynb

<ipython-input-126-11ba52ca5917>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['company']=df.apply(lambda x: getCompanyName(x.sponsor_collaborators, x.lead_sponsor, x.collaborators), axis=1)
<ipython-input-126-11ba52ca5917>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['symbol']=df.company.apply(lambda x: findSymbol(x, ticker_all))


In [127]:
# temporary turn on debug
debug = True

In [128]:
# question related source data overview
if (debug):
    print('source data shape: ', df.shape)
    print('total unique symbols:', df.symbol.nunique())
    print('symbol list', df.symbol.unique())

source data shape:  (367, 70)
total unique symbols: 152
symbol list ['PLRX' 'PPD' 'NVS' 'BCRX' 'GILD' 'REGN' 'SNY' 'DGX' 'VNDA' 'ABBV' 'PFE'
 'IFRX' 'CANF' 'BLDE' 'INO' 'CAPR' 'TRHC' 'CDNA' 'IMAB' 'LLY' 'EIGR' 'AZN'
 'BHVN' 'BMY' 'KPTI' 'TBPH' 'HGEN' 'UNH' 'INCY' 'ALXN' 'BHC' 'CNTG' 'MRKR'
 'BLPH' 'LBPS' 'MEDP' 'MESO' 'ATHX' 'IQV' 'BNTX' 'NVAX' 'IPHA' 'INMB'
 'VSTM' 'JNJ' 'GSK' 'IMUX' 'AIM' 'ECOR' 'BGNE' 'IBRX' 'VERU' 'PSTI' 'WINT'
 'CMRX' 'CTSO' 'EXAS' 'AVIR' 'XAIR' 'KNSA' 'ALVR' 'EDSA' 'HOPE' 'APLS'
 'VTGN' 'CTIC' 'CLOV' 'PTNR' 'LIFE' 'KLDO' 'RDHL' 'PHAR' 'SNDX' 'TAK'
 'DVA' 'MNOV' 'FGEN' 'PHAS' 'PTCT' 'SRNE' 'ALT' 'DRRX' 'CVAC' 'AMPE'
 'ANGN' 'AMRN' 'CRVS' 'VAR' 'BMRA' 'HRTX' 'BPTS' 'GRFS' 'ARCT' 'EVLO'
 'HAE' 'RETA' 'ADPT' 'VIR' 'MMM' 'ARPO' 'FULC' 'SNN' 'CLBS' 'GLBL' 'VRNA'
 'RDY' 'JAZZ' 'SAGE' 'MREO' 'IONS' 'KMDA' 'OPK' 'GRTX' 'EW' 'ILMN' 'VXRT'
 'DFFN' 'HRC' 'RIGL' 'NGM' 'AMGN' 'AEMD' 'CFRX' 'NRBO' 'EQ' 'CLNN' 'HUGE'
 'CERT' 'ONCS' 'LGVN' 'NKTR' 'ICLR' 'PRTC' 'ABIO' 'ENLV' 'EBS'

In [129]:
# get unique symbol and max time period to download price data
symbols = df[['symbol','last_update_posted']].groupby('symbol').agg(
    mindate=('last_update_posted','min'),
    maxdate=('last_update_posted','max'),
)
symbols = symbols.reset_index()

In [133]:
# functions to download symbol price during a period
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
def yfdownload(symbol, startdate, enddate):
    try:
        df = yf.download(symbol,start=startdate, end=enddate, progress=False, auto_adjust=True)
    except: 
        df = pd.DataFrame()
    return df

def getSymbolPriceData(symbol, mindate, maxdate):    
    startdate = datetime.strptime(mindate, "%Y-%m-%d") - timedelta(days=7) # one week before
    enddate = datetime.strptime(maxdate, "%Y-%m-%d") + timedelta(weeks=4)  # one month after
    #print('symbol:', symbol,' startdate:', startdate, ' enddate:', enddate)
    
    # some company went public after clinical trial, so no price for some dates
    for single_date in daterange(startdate, enddate):
        #print(single_date.strftime("%Y-%m-%d"))
        df = yfdownload(symbol,single_date, enddate)
        #print('downloaded data shape:',df.shape)
        if (df.shape[0]>0):
            # add symbol back to the data frame
            df['symbol']=symbol
            break
    
    return df

In [131]:
# getSymbolPriceData testing:
if (debug):
    print(getSymbolPriceData('ABC','2020-02-10','2020-02-10'))

2020-02-03


In [134]:
# download all symbol price data and save for late use since we may hit the daily free download limits
allprice = pd.DataFrame()
for row in symbols.itertuples(name='ticker'):
    allprice = allprice.append(getSymbolPriceData(row.symbol, row.mindate, row.maxdate))

allprice.to_csv('allprice' + str(datetime.now())[-7:-1] + '.csv')

In [135]:
## errors from yf:
# - ANGN: Data doesn't exist for startDate = 1598760000, endDate = 1598932800 (2020-07-29  2020-08-31)
# - CERT: Data doesn't exist for startDate = 1604552400, endDate = 1607576400 (2020-11-06  2020-12-09)
# - GLBL: Data doesn't exist for startDate = 1612328400, endDate = 1615352400 (2021-02-04  2021-03-09)
# - LBPS: Data doesn't exist for startDate = 1592971200, endDate = 1595995200 (2020-06-25  2020-07-28)
# - LGVN: Data doesn't exist for startDate = 1604898000, endDate = 1607922000 (2020-11-10  2020-12-13)
# - VALN: Data doesn't exist for startDate = 1614661200, endDate = 1617681600 (2021-03-03  2021-04-05)


In [136]:
# find out more about symbols has no price
if (debug):
    print('all symbol price data shape:', allprice.shape)
    tickers = set(df.symbol.unique()) - set(allprice.symbol.unique())
    print('no price symbols: ', tickers)
    
    tickerlist = ''
    for ticker in tickers:
        tickerlist = tickerlist + ' ' + ticker
        #hist = ticker.history(period="max")
        #print(hist)
    
    #tickers = yf.Tickers("ANGN CERT GLBL LBPS LGVN VALN" ) #['BPTS', 'CVAC']
    tickers = yf.Tickers(tickerlist.strip()) 
    
    hist=tickers.tickers['ANGN'].history(period="max").head(1)
    # hist=tickers.tickers['BPTS'].history(period="max").head(1)
    # hist=tickers.tickers['CVAC'].history(period="max").head(1)
    # hist=tickers.tickers['LBPS'].history(period="max").head(1)
    print(hist)

all symbol price data shape: (8835, 7)
no price symbols:  {'CERT', 'ANGN', 'LBPS', 'VALN', 'LGVN', 'GLBL'}
                 Open       High   Low  Close   Volume  Dividends  \
Date                                                                
2021-02-05  17.620001  18.985001  16.5  17.02  1667100          0   

            Stock Splits  
Date                      
2021-02-05             0  


In [142]:
allprice = allprice.reset_index()
allprice.dtypes

index                 int64
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Volume              float64
symbol               object
Adj Close           float64
dtype: object

In [143]:
allprice.to_csv('allprice' + str(datetime.now())[-7:-1] + '.csv')